In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import torch
import torchtext
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

import pandas as pd
import re
from collections import Counter
from tqdm.notebook import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 能用gpu则用gpu

# 1.准备数据

In [2]:
class MyDataset(Dataset):
    def __init__(self, file_path, tokenizer, stopwords, debug=True):
        df = pd.read_csv(file_path)
        df = df.dropna().reset_index(drop=True)
        if debug:
            df = df.sample(2000).reset_index(drop=True)
        counter = Counter()
        sentences = []
        for title in tqdm(df['title']):
            # 去除标点符号
            title = re.sub(r'[^\u4e00-\u9fa5]', '', title)
            tokens = [token for token in tokenizer(title.strip()) if token not in stopwords]
            counter.update(tokens)
            sentences.append(tokens)
        self.vocab = torchtext.vocab.vocab(counter, specials=['<unk>', '<pad>'])
        self.vocab.set_default_index(self.vocab['<unk>'])

        self.inputs = [self.vocab.lookup_indices(tokens) for tokens in sentences]
        self.labels = [[label] for label in df['label'].values.tolist()]
        self.n_class = len(df['label'].unique())

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.LongTensor(self.inputs[idx]), torch.LongTensor(self.labels[idx])

In [3]:
file_path = '../data/THUCNews/train.csv'
tokenizer = torchtext.data.utils.get_tokenizer('spacy', language='zh_core_web_sm')
stopwords = [line.strip() for line in open('../stopwords/cn_stopwords.txt', 'r', encoding='utf-8').readlines()]
dataset = MyDataset(file_path, tokenizer, stopwords, debug=False)

  0%|          | 0/501644 [00:00<?, ?it/s]

In [4]:
def collate_fn(batch_data):
    return pad_sequence([x for x, y in batch_data], padding_value=1), torch.tensor([y for x, y in batch_data]).unsqueeze(1)

dataloader = DataLoader(dataset, batch_size=1024, shuffle=True, collate_fn=collate_fn)

# 2.构建模型

In [5]:
class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_size, n_class):
        super().__init__()
        # 嵌入层
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # 输入通道为1，卷成16通道输出，卷积核大小为(3*embed_size)，3类似于n-gram，可以换
        self.conv = nn.Conv2d(1, 16, (3, embed_size))
        self.dropout = nn.Dropout(0.2)
        # 输出头
        self.fc = nn.Linear(16, n_class)

    def forward(self, x):  # x: [batch_size * 句子长度]
        x = x.permute(1, 0)
        x = self.embedding(x)  # [batch_size * 句子长度 * embed_size]
        x = x.unsqueeze(1)  # [batch_size * 1 * 句子长度 * embed_size]，加一个维度，用于卷积层的输入
        x = self.conv(x)  # [batch_size * 16(卷积层输出通道数) * 8(卷积后的宽) * 1(卷积后的高)]
        x = x.squeeze(3)  # [batch_size * 16(卷积层输出通道数) * 8(卷积后的宽)] 压缩大小为1的维度
        x = torch.relu(x)  # 激活函数，尺寸不变
        x = torch.max_pool1d(x, x.size(2))  # 在每个通道做最大池化，[batch_size * 16(卷积层输出通道数) * 1]
        x = x.squeeze(2)  # 压缩维度2，[batch_size * 16(卷积层输出通道数)]
        x = self.dropout(x)  # dropout，尺寸不变
        logits = self.fc(x)  # 全连接输出头，[batch_size * n_class]
        return logits


model = TextCNN(vocab_size=len(dataset.vocab), embed_size=256, n_class=dataset.n_class).to(device)
print(model)

TextCNN(
  (embedding): Embedding(332814, 256)
  (conv): Conv2d(1, 16, kernel_size=(3, 256), stride=(1, 1))
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=16, out_features=14, bias=True)
)


# 3.训练模型

In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
model.train()
for epoch in range(20):
    for feature, target in dataloader:
        feature = feature.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        logits = model(feature)
        loss = criterion(logits, target.squeeze())
        loss.backward()
        optimizer.step()
    print('epoch:', epoch + 1, ', loss:', loss.item())

epoch: 1 , loss: 1.2081621885299683
epoch: 2 , loss: 0.8597992658615112
epoch: 3 , loss: 0.6249662041664124
epoch: 4 , loss: 0.566360354423523
epoch: 5 , loss: 0.5031101107597351
epoch: 6 , loss: 0.48120951652526855
epoch: 7 , loss: 0.45753684639930725
epoch: 8 , loss: 0.38434842228889465
epoch: 9 , loss: 0.3268248438835144
epoch: 10 , loss: 0.2881065309047699
epoch: 11 , loss: 0.307202011346817
epoch: 12 , loss: 0.26004788279533386
epoch: 13 , loss: 0.25084903836250305
epoch: 14 , loss: 0.2770867943763733
epoch: 15 , loss: 0.22041885554790497
epoch: 16 , loss: 0.19741909205913544
epoch: 17 , loss: 0.2005310356616974
epoch: 18 , loss: 0.18645475804805756
epoch: 19 , loss: 0.2028864324092865
epoch: 20 , loss: 0.19458307325839996


# 4.预测

In [9]:
model.eval()
df_train = pd.read_csv('../data/THUCNews/train.csv')
df_test = pd.read_csv('../data/THUCNews/test.csv')

In [10]:
for i, row in df_test.sample(10).iterrows():
    title = row['title']
    actual = row['class']
    title = re.sub(r'[^\u4e00-\u9fa5]', '', title)
    tokens = [token for token in tokenizer(title.strip()) if token not in stopwords]
    inputs = dataset.vocab.lookup_indices(tokens)
    inputs = torch.LongTensor(inputs).unsqueeze(1).to(device)
    predict = model(inputs)
    predict_class = dict(zip(df_train['label'], df_train['class']))[predict.max(1)[1].item()]    
    print(' '.join(tokens), '||| actual:', actual, ', predict:', predict_class)

灰熊 闪电 宣布 新 主帅 昔日 冠军队 球员 接手 教鞭 ||| actual: 体育 , predict: 体育
香港 增加 未能 如期 交收 证券 罚款 ||| actual: 股票 , predict: 股票
鲁能 转会 挂牌 政策 独树 中 超 铁板凳 用武之地 ||| actual: 体育 , predict: 体育
老人 出院 儿子 关在 家门 外 小时 图 ||| actual: 社会 , predict: 社会
收储 结束 产能 重启 空头 压制 沪 铝 ||| actual: 财经 , predict: 股票
韩国 电讯 苹果 洽谈 引进 ||| actual: 科技 , predict: 科技
年 甘肃省 在职 人员 硕士 联考 岁考 生来 赶考 ||| actual: 教育 , predict: 教育
瑞宝 壁纸 价格 统一 服务 统一 保障 统一 树 行业 标杆 ||| actual: 家居 , predict: 家居
规划 餐厅 区域 展示机 图 ||| actual: 家居 , predict: 家居
餐厅 鲜花 美食 精致 情调 图 ||| actual: 家居 , predict: 家居
